In [1]:
from cuml.svm import SVC, LinearSVC
import cudf
#from cuml.model_selection import train_test_split
import cupy as cp
from cuml.metrics import accuracy_score
from cuml.pipeline import make_pipeline
from cuml.decomposition import PCA

from sklearn.datasets import fetch_openml
import pandas as pd
from sklearn.model_selection import GridSearchCV, train_test_split
#from sklearn.svm import SVC
#from sklearn.metrics import accuracy_score
import numpy as np
from itertools import product
#from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from matplotlib.lines import Line2D
from sklearn.metrics import confusion_matrix
import seaborn as sns

In [2]:
data = fetch_openml('mnist_784', version=1)

Xdata, ydata = data.data, data.target
Xdata = Xdata.astype(np.float32)
ydata = ydata.astype(np.float32)

# Xdata = Xdata[:200]
# ydata = ydata[:200]

Xtrain, Xtemp, ytrain, ytemp = train_test_split(Xdata, ydata, test_size=0.3, random_state=42, stratify=ydata)
Xval, Xtest, yval, ytest = train_test_split(Xtemp, ytemp, test_size=0.4, random_state=42, stratify=ytemp)

Xtrain_cp = cp.asarray(Xtrain)
ytrain_cp = cp.asarray(ytrain)
Xval_cp = cp.asarray(Xval)
yval_cp = cp.asarray(yval)

In [3]:
# hyperparameters

grid_params = {
    'C': [0.1, 1, 10],
    'gamma': ['scale', 'auto', 0.01],
    'kernel': ['rbf', 'linear', 'poly', 'sigmoid']
}

In [4]:
lda = LDA(n_components=len(ydata.unique())-1)  # Reduce to 9 components (since we have 10 classes)
Xtrain_lda = lda.fit_transform(Xtrain, ytrain)
Xval_lda = lda.transform(Xval)
Xtest_lda = lda.transform(Xtest)

In [5]:
Xtrain_lda_cp = cp.asarray(Xtrain_lda)
Xval_lda_cp = cp.asarray(Xval_lda)
Xtest_lda_cp = cp.asarray(Xtest_lda)

In [6]:
best_model_lda = None
best_params_lda = None
best_val_acc_lda = 0
val_accs_lda = []

In [7]:
param_combinations_lda = product(grid_params['C'], grid_params['gamma'], grid_params['kernel'])

for C, gamma, kernel in param_combinations_lda:
    if kernel == 'linear':
        svm = SVC(C=C)
    else:
        svm = SVC(C=C, gamma=gamma, kernel=kernel)
    
    svm.fit(Xtrain_lda_cp, ytrain_cp)
    pred = svm.predict(Xval_lda_cp)
    val_acc = accuracy_score(yval, pred)
    val_accs_lda.append(val_acc)
    #print(f"Params: C={C}, gamma={gamma}, kernel={kernel} | Evaluation Accuracy: {val_acc:.4f}")
    
    if val_acc > best_val_acc_lda:
        best_val_acc_lda = val_acc
        best_model_lda = svm
        best_params_lda = {'C': C, 'gamma': gamma, 'kernel': kernel}

print("Best hyperparameters:", best_params_lda)
print("HIghest validation accuracy:", best_val_acc_lda)

Best hyperparameters: {'C': 1, 'gamma': 'auto', 'kernel': 'rbf'}
HIghest validation accuracy: 0.925000011920929
